In [9]:
import pandas as pd
import re
import numpy as np
import multiprocessing
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
%matplotlib inline

import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding
from keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report, roc_curve, auc
import string
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import warnings
warnings.filterwarnings('ignore')

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
import pandas as pd

# Path to your CSV file
csv_file_path = '/content/drive/MyDrive/HateSpeechDataset.csv'

# Load the CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)
df.head()

,Content,Label,Content_int
0,denial of normal the con be asked to comment o...,1,"[146715, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11,..."
1,just by being able to tweet this insufferable ...,1,"[146715, 14, 15, 16, 17, 7, 18, 19, 20, 21, 22..."
2,that is retarded you too cute to be single tha...,1,"[146715, 28, 29, 30, 26, 31, 32, 7, 5, 33, 28,..."
3,thought of a real badass mongol style declarat...,1,"[146715, 35, 1, 24, 36, 37, 38, 39, 40, 1, 41,..."
4,afro american basho,1,"[146715, 46, 47, 48, 146714]"


In [14]:
train = df.drop(['Content_int'], axis = 1)

In [16]:
import nltk
import string
import re

# Download the 'stopwords' resource
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words('english'))
punctuation = list(string.punctuation)
stop.update(punctuation)



#Removing the stopwords from text
def split_into_words(text):
    # split into words by white space
    words = text.split()
    return words

def to_lower_case(words):
    # convert to lower case
    words = [word.lower() for word in words]
    return words

def remove_punctuation(words):
    # prepare regex for char filtering
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    stripped = [re_punc.sub('', w) for w in words]
    return stripped

def keep_alphabetic(words):
    # remove remaining tokens that are not alphabetic
    words = [word for word in words if word.isalpha()]
    return words

def remove_stopwords(words):
    # filter out stop words
    stop_words = set(stopwords.words('english'))
    words = [w for w in words if not w in stop_words]
    return words

def to_sentence(words):
    # join words to a sentence
    return ' '.join(words)
def tweet(words):
    tweet_tokenizer = nltk.tokenize.TweetTokenizer(strip_handles=True,reduce_len=True)
    tweet = tweet_tokenizer.tokenize(words)
    return tweet

#Removing the noisy text
def denoise_text(text):
    words = split_into_words(text)
    words = to_lower_case(words)
    words = remove_punctuation(words)
    words = keep_alphabetic(words)
    words = remove_stopwords(words)
    return to_sentence(words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [17]:
train['Content']=train['Content'].apply(denoise_text)

In [18]:
train

,Content,Label
0,denial normal con asked comment tragedies emot...,1
1,able tweet insufferable bullshit proves trump ...,1
2,retarded cute single life,1
3,thought real badass mongol style declaration w...,1
4,afro american basho,1
...,...,...
440901,crash another movie left field ask star wars e...,0
440902,debate first starting edit war,0
440903,removal reverted removal section believe misgu...,0
440904,unblocked eddie discussed user talkable accusi...,0


In [22]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

X = train.Content
Y = train.Label
le = LabelEncoder() # Now LabelEncoder is defined correctly
Y = le.fit_transform(Y)
Y = Y.reshape(-1,1)

In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.15)

In [28]:
from keras import utils
from keras.preprocessing.text import Tokenizer
max_words = 1000
max_len = 200
tok = Tokenizer(num_words=max_words)
tok.fit_on_texts(X_train)
sequences = tok.texts_to_sequences(X_train)
sequences_matrix = utils.pad_sequences(sequences,maxlen=max_len)

In [37]:
from keras.models import Model
from keras.layers import Input, Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout, Activation
from keras.callbacks import EarlyStopping # Import EarlyStopping
def CNN():
    inputs = Input(name='inputs', shape=[max_len])
    layer = Embedding(max_words, 50, input_length=max_len)(inputs)
    layer = Conv1D(filters=64, kernel_size=3, padding='same', activation='relu')(layer)
    layer = MaxPooling1D(pool_size=2)(layer)
    layer = Conv1D(filters=128, kernel_size=3, padding='same', activation='relu')(layer)
    layer = MaxPooling1D(pool_size=2)(layer)
    layer = Flatten()(layer)
    layer = Dense(256, name='FC1')(layer)
    layer = Activation('relu')(layer)
    layer = Dropout(0.5)(layer)
    layer = Dense(1, name='out_layer')(layer)
    layer = Activation('sigmoid')(layer)
    model = Model(inputs=inputs, outputs=layer)
    return model


In [38]:

model = CNN()
model.summary()
#model.compile(loss='binary_crossentropy',optimizer=RMSprop(),metrics=['accuracy'])

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inputs (InputLayer)         [(None, 200)]             0         
                                                                 
 embedding_4 (Embedding)     (None, 200, 50)           50000     
                                                                 
 conv1d_8 (Conv1D)           (None, 200, 64)           9664      
                                                                 
 max_pooling1d_8 (MaxPoolin  (None, 100, 64)           0         
 g1D)                                                            
                                                                 
 conv1d_9 (Conv1D)           (None, 100, 128)          24704     
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, 50, 128)           0         
 g1D)                                                      

In [42]:
from keras.callbacks import EarlyStopping

# Assuming you have already defined the CNN model as given earlier
model = CNN()

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Define early stopping
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=3, restore_best_weights=True)

# Train the model
history = model.fit(sequences_matrix, Y_train,
                    batch_size=128,
                    epochs=10,
                    validation_split=0.2,
                    callbacks=[early_stopping])


Epoch 1/10
2343/2343 [==============================] - 443s 188ms/step - loss: 0.3544 - accuracy: 0.8403 - val_loss: 0.3423 - val_accuracy: 0.8483
Epoch 2/10
2343/2343 [==============================] - 453s 193ms/step - loss: 0.3320 - accuracy: 0.8511 - val_loss: 0.3342 - val_accuracy: 0.8512
Epoch 3/10
2343/2343 [==============================] - 430s 184ms/step - loss: 0.3191 - accuracy: 0.8564 - val_loss: 0.3338 - val_accuracy: 0.8519
Epoch 4/10
2343/2343 [==============================] - 454s 194ms/step - loss: 0.3057 - accuracy: 0.8613 - val_loss: 0.3370 - val_accuracy: 0.8528
Epoch 5/10
2343/2343 [==============================] - 434s 185ms/step - loss: 0.2930 - accuracy: 0.8665 - val_loss: 0.3469 - val_accuracy: 0.8503
Epoch 6/10
2343/2343 [==============================] - 450s 192ms/step - loss: 0.2804 - accuracy: 0.8717 - val_loss: 0.3519 - val_accuracy: 0.8533


In [45]:
from keras import utils
from keras.preprocessing.text import Tokenizer

# ... (Your existing code for defining 'tok', 'max_len', and 'max_words')

# Preprocess the test data
test_sequences = tok.texts_to_sequences(X_test)  # Assuming X_test is your test data
test_sequences_matrix = utils.pad_sequences(test_sequences, maxlen=max_len)

# ... (Rest of your code)

# Evaluate the model
accr = model.evaluate(test_sequences_matrix, Y_test)

2067/2067 [==============================] - 34s 16ms/step - loss: 0.3347 - accuracy: 0.8516


In [46]:
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

Test set
  Loss: 0.335
  Accuracy: 0.852
